## Basics: classes, instances, class vs instance variables

In [27]:
class Blog(object):
    """Good practice (py3) is to explicitly inherit from object"""

    # class variable = shared among instances
    num_blogs = 0

    def __init__(self, name, bio):
        """The constructor, gets called implicitly when instantiating a new object (next cell)"""
        self.name = name
        self.bio = bio
        Blog.num_blogs += 1  # we access a class variable like this (or use a class method)
    
    def get_articles(self, rss):
        """Get all articles from RSS"""
        print('Articles {}: ...'.format(rss))

    def post_article(self, title):
        """Add a new article"""
        print('Posted new article: {}'.format(title))
    
    def __str__(self):
        """Informal or nicely printable string representation of an object"""
        return '{}: {}'.format(self.name, self.bio)

In [28]:
name = 'PyBites'
bio = 'Python code challenges, tutorials and news, one bite a day'
blog = Blog(name, bio)

In [29]:
blog

In [30]:
print(blog)  # invokes __str__

PyBites: Python code challenges, tutorials and news, one bite a day


In [31]:
blog.get_articles('http://pybit.es/feeds/all.rss.xml')

Articles http://pybit.es/feeds/all.rss.xml: ...


In [32]:
blog.post_article('OOP fun with PyBites')

Posted new article: OOP fun with PyBites


In [33]:
print(blog.num_blogs)

1


In [34]:
blog2 = Blog('bobcodesit', 'my personal blog')

In [35]:
blog2

In [36]:
blog == blog2

False

In [37]:
print(blog.num_blogs)

2


## Abstract base classes (ABC's)

Let's create some developers to add to a blog.

ABC's lets you force derived classes to implement certain behaviors / methods (tip from Dan Bader)

In [38]:
from abc import ABCMeta, abstractmethod

class Developer(metaclass=ABCMeta):
    
    @abstractmethod
    def get_post_days(self):
        """Classes that inherit from Developer need to implement this method"""
        pass
    
    def __str__(self):
        return self.__class__.__name__

In [39]:
class Julian(Developer):
    pass

# oops, did not implement get_post_days()
jul = Julian()

TypeError: Can't instantiate abstract class Julian with abstract methods get_post_days

In [40]:
class Julian(Developer):
    
    def get_post_days(self):
        return 'Tue Wed'.split()

# ok
julian = Julian()

In [41]:
class Bob(Developer):
    
    def get_post_days(self):
        return 'Wed Thurs'.split()
    
bob = Bob()

In [42]:
class PyBites(Developer):
    
    def get_post_days(self):
        return 'Mon Fri Sat'.split()

pybites = PyBites()

## Inheritance and using objects inside objects

In [43]:
class PyBitesBlog(Blog):
    """PyBitesBlog inherits from parent class Blog"""
    
    def __init__(self, *args):
        """We get a variable-length argument list of developers"""
        self.name = self.__class__.__name__  # how to get name of class 'PyBitesBlog' as string
        self.bio = 'Python code challenges, tutorials and news, one bite a day'
        self.developers = args
        # pass name and bio to the parent class
        # py2 requires longer syntax: super(PyBitesBlog, self).__init__
        super().__init__(self.name, self.bio)
        
    def schedule(self):
        """Loop over all developer objects calling their get_post_days method"""
        for dev in self.developers:
            print('{}: {}'.format(dev, ', '.join(dev.get_post_days())))
            
    def __str__(self):
        """Use the parent __str__ adding something extra"""
        return super().__str__() + '\nAuthors: ' + ', '.join([str(dev) for dev in self.developers])

In [44]:
pyblog = PyBitesBlog(julian, bob, pybites)

In [46]:
print(pyblog)  # again call object's __str__

PyBitesBlog: Python code challenges, tutorials and news, one bite a day
Authors: Julian, Bob, PyBites


In [47]:
pyblog.get_articles('http://pybit.es/feeds/all.rss.xml')  # inherited method from parent

Articles http://pybit.es/feeds/all.rss.xml: ...


In [48]:
pyblog.schedule()

Julian: Tue, Wed
Bob: Wed, Thurs
PyBites: Mon, Fri, Sat


and  this is ...

## Polymorphism

We call get_post_days() on every developer (instance), each one giving a different result. 

Good examples from stdlib are len (= \_\_len\_\_()) and + (= \_\_add\_\_()): they just do what you expect them to do for each type (as long as compatible):

In [49]:
s = 'a string'
t = (0, 1)
d = dict(zip([1,2,3], ['a','b','c']))

In [50]:
for var in (s, t, d): 
    print('len of {} (type {}) = {}'.format(var, type(var), len(var)))

len of a string (type <class 'str'>) = 8
len of (0, 1) (type <class 'tuple'>) = 2
len of {1: 'a', 2: 'b', 3: 'c'} (type <class 'dict'>) = 3


In [54]:
s = ('python', '123')
i = (3, 6)
lst = [1, 2, 3], [4, 5]

for var in (s, i, lst):
    res = var[0] + var[1]
    print('calling + on {} yields {} (type {})'.format(var, res, type(res)))

calling + on ('python', '123') yields python123 (type <class 'str'>)
calling + on (3, 6) yields 9 (type <class 'int'>)
calling + on ([1, 2, 3], [4, 5]) yields [1, 2, 3, 4, 5] (type <class 'list'>)


In [55]:
# ok cannot add str + int
'a' + 1

TypeError: Can't convert 'int' object to str implicitly

In [56]:
# but this works
'a' + str(1)

'a1'

## Encapsulation

http://stackoverflow.com/questions/26216563/how-to-do-encapsulation-in-python

> Python has encapsulation - you are using it in your class.
> 
> What it doesn't have is access control such as private and protected attributes. However, in Python, there is an attribute naming convention to denote private attributes by prefixing the attribute with one or two underscores, e.g:
> 
> self._a
> self.__a 
> A single underscore indicates to the user of a class that an attribute should be considered private to the class, and should not be accessed directly.
> 
> A double underscore indicates the same, however, Python will mangle the attribute name somewhat to attempt to hide it.

If you want getter / setter behavior look at the @property decorator:

http://stackabuse.com/python-properties/

> It is often considered best practice to create getters and setters for a class's public properties. Many languages allow you to implement this in different ways, either by using a function (like person.getName()), or by using a language-specific get or set construct. In Python, it is done using @property.
